## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
# %autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.models.cifar10.preact_resnet import *
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10/")
os.makedirs(PATH,exist_ok=True)

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

bs=512
sz=32
workers=7

In [4]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def pad(img, p=4, padding_mode='reflect'):
        return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))
def to_pil(img): return Image.fromarray(img)

def torch_tfms(size, conv_pil=False, to_numpy=False):
    mean,std=[0.4914 , 0.48216, 0.44653], [0.24703, 0.24349, 0.26159]
    normalize = transforms.Normalize(mean=mean, std=std)
    tfms = [transforms.ToTensor(), normalize]
    
    # Torch transforms with fastai dl
    if to_numpy: tfms = [np.array, Normalize(mean,std), lambda x: x[0].T]
        
    aug_tfms = [
        pad, # TODO: use `padding` rather than assuming 4
        transforms.RandomCrop(size),
        transforms.ColorJitter(.25,.25,.25),
        transforms.RandomRotation(2),
        transforms.RandomHorizontalFlip(),
    ]
    scale_size = 40
    padding = int((scale_size - size) / 2)
    
    train_tfms = transforms.Compose(aug_tfms + tfms)
    train_tfms.sz = size
    val_tfms = transforms.Compose(tfms)
    val_tfms.sz = size
    if conv_pil:
        train_tfms.transforms.insert(0, to_pil)
        val_tfms.transforms.insert(0, to_pil)
    return train_tfms, val_tfms
    
def torch_ds(data_path, tfms):
    train_tfms, val_tfms = tfms
    
    # Data loading code
    traindir = os.path.join(data_path, 'train')
    valdir = os.path.join(data_path, 'test')

    train_dataset = datasets.ImageFolder(traindir, train_tfms)
    val_dataset = datasets.ImageFolder(valdir, val_tfms)
    return train_dataset, val_dataset

def torch_loader(data_path, datasets):
    train_dataset, val_dataset = datasets
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs*2, shuffle=False,
        num_workers=workers, pin_memory=True)

    data = ModelData(data_path, train_loader, val_loader)
    return data


In [5]:
def get_learner(data):
    m = PreActResNet18()
    learn = ConvLearner.from_model_data(m, data)
    learn.crit = nn.CrossEntropyLoss()
    learn.metrics = [accuracy]
    learn.half()
    return learn

### Torch Data Loader

In [6]:
tfms = torch_tfms(sz)
dsets = torch_ds(PATH, tfms)
torch_data = torch_loader(str(PATH), dsets)

In [7]:
learn = get_learner(torch_data)

In [8]:
# Warmup session
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                
    0      nan        nan        0.1       

CPU times: user 5.76 s, sys: 2.74 s, total: 8.5 s
Wall time: 11.4 s


[array([nan]), 0.1]

In [9]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 10.1 s, sys: 3.98 s, total: 14.1 s
Wall time: 15.8 s


[array([nan]), 0.1]

### Fastai Data Loader

In [10]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [11]:
learn = get_learner(fastai_data)

In [12]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 1min 48s, sys: 2min 51s, total: 4min 39s
Wall time: 46.9 s


[array([nan]), 0.1]

### Fastai Default Collate

In [ ]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_dc_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [ ]:
# Hack to get the hidden collate function out of the torch dataloader class. 
collate_fn = torch.utils.data.DataLoader(None).collate_fn

In [ ]:
fastai_dc_data.trn_dl.collate_fn = collate_fn
fastai_dc_data.val_dl.collate_fn = collate_fn

In [ ]:
learn = get_learner(fastai_dc_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

### Fastai DataLoader - Torch Transforms

In [ ]:
tfms = torch_tfms(sz, to_numpy=True)

In [ ]:
fdl_tds_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [ ]:
fdl_tds_data.trn_ds.open_fn = Image.open
fdl_tds_data.val_ds.open_fn = Image.open

In [ ]:
learn = get_learner(fdl_tds_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

### Torch DataLoader - Fastai Datasets

In [15]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fdl = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)
tdl_fds = fdl.trn_ds, fdl.val_ds
torch_data = torch_loader(str(PATH), tdl_fds)

In [16]:
learn = get_learner(torch_data)

In [17]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 9.9 s, sys: 4.5 s, total: 14.4 s
Wall time: 15 s


[array([nan]), 0.1]

### Fastai DataLoader - Torch Dataset - Same as Torch Transforms

In [ ]:
# from fastai.dataset import FilesArrayDataset, folder_source, read_dir

In [45]:
tfms = torch_tfms(sz, to_numpy=True)
dsets = torch_ds(PATH, tfms)

In [46]:
fdl_tds_data = ImageData(PATH, list(dsets)+[None,None,None,None], bs, num_workers=workers, classes=dsets[0].classes)

In [47]:
learn = get_learner(fdl_tds_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

### Fastai DataLoader - Torch Dataset - Same as Torch Transforms

In [6]:
tfms = torch_tfms(sz, to_numpy=True)
dsets = torch_ds(PATH, tfms)
torch_data = torch_loader(str(PATH), dsets)

In [7]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fdl = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)
# Hack to get the hidden collate function out of the torch dataloader class. 
collate_fn = fdl.trn_dl.np_collate

In [8]:
torch_data.trn_dl.collate_fn = collate_fn
torch_data.val_dl.collate_fn = collate_fn

In [9]:
learn = get_learner(torch_data)

In [19]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

  1%|          | 1/98 [00:00<01:14,  1.30it/s, loss=nan]

RuntimeError: cuda runtime error (77) : an illegal memory access was encountered at torch/csrc/cuda/Module.cpp:267

### ProcessPoolExecutor

In [13]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [14]:
fastai_data.trn_dl.multiprocess = True
fastai_data.val_dl.multiprocess = True

In [15]:
learn = get_learner(fastai_data)

In [16]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 38.3 s, sys: 1min 8s, total: 1min 46s
Wall time: 18.9 s


[array([nan]), 0.1]

In [52]:


class TestDataLoader(object):
    def __init__(self):
        super().__init__()
        self.batch_sampler = range(10)

    def get_batch(self, indices):
        return [indices]

    def __iter__(self):
        with ProcessPoolExecutor(max_workers=8) as e:
            # avoid py3.6 issue where queue is infinite and can result in memory exhaustion
            for batch in e.map(self.get_batch, iter(self.batch_sampler)): 
                yield batch


In [53]:
tdl = TestDataLoader()

In [54]:
itdl = iter(tdl)

In [56]:
next(itdl)

[1]

In [30]:
class Test():
    def __init__(self):
        pass

In [31]:
a = Test()

In [32]:
a.hey = 'test'

In [33]:
a.hey

'test'